In [0]:
!pip3 install tensorflow_gpu==2.0


import tensorflow as tf
print(tf.__version__)

2.0.0


In [0]:

!pip3 install -q ktrain

In [0]:
import pandas as pd
import ktrain
from ktrain import text
import numpy as np
from sklearn.model_selection import train_test_split

using Keras version: 2.2.4-tf


In [0]:
data = pd.read_csv('/content/drive/My Drive/Máster KSchool/TFM/data/filtered_df_with_imgNames.csv',usecols=['RATING','TITLE'])
data = data.apply(lambda x: x.str.replace(',','.'))

In [0]:
data

,TITLE,RATING
0,Love Beats Rhymes,5.3
1,Mass,6.4
2,Vicky el vikingo,5.5
3,Vuelven,7.0
4,Timecode,6.0
...,...,...
107176,Jen o rodinnÃƒÂ½ch zÃƒÂ¡lezitostech,7.3
107177,The Angry Red Planet,5.3
107178,Fantasmas en la casa,5.1
107179,Laughing Boy,5.5


In [0]:
data['RATING']  = data['RATING'].astype(float)
data['RATING'] = pd.cut( data['RATING'], bins = [0, 5, 6.5, 10], labels=['0','1','2'])
data

,TITLE,RATING
0,Love Beats Rhymes,1
1,Mass,1
2,Vicky el vikingo,1
3,Vuelven,2
4,Timecode,1
...,...,...
107176,Jen o rodinnÃƒÂ½ch zÃƒÂ¡lezitostech,2
107177,The Angry Red Planet,1
107178,Fantasmas en la casa,1
107179,Laughing Boy,1


In [0]:
print(len(data[data['RATING']=='2']))
print(len(data[data['RATING']=='1']))
print(len(data[data['RATING']=='0']))

40270
47141
19770


In [0]:
MODEL_NAME = 'bert-base-multilingual-uncased'
classes=['0','1','2']
t = text.Transformer(MODEL_NAME, maxlen=500, classes=classes)

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:324: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


In [0]:
X = data.TITLE.values
Y = data.RATING.values

In [0]:
#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
x_train, x_test, y_train, y_test= train_test_split(X, Y.astype(str), stratify=Y, test_size=0.2)

In [0]:
x_train = list(x_train)
x_test = list(x_test)

In [0]:
y_train = np.array(y_train).astype(int)
y_test = np.array(y_test).astype(int)

In [0]:
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 3
	95percentile : 6
	99percentile : 8


preprocessing test...
language: en
test sequence lengths:
	mean : 3
	95percentile : 6
	99percentile : 8


In [0]:
model = t.get_model()
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  167356416 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 167,358,723
Trainable params: 167,358,723
Non-trainable params: 0
_________________________________________________________________


In [0]:
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=128)

In [0]:
### To freeze Distilbert weights
dBert = model.layers[0]

dBert.trainable = False

In [0]:
learner.fit_onecycle(5e-5, 100)



begin training using onecycle policy with max lr of 5e-05...
Train for 670 steps, validate for 168 steps
Epoch 1/100
595/670 [=========================>....] - ETA: 3:44 - loss: 1.0629 - accuracy: 0.4009

In [0]:
learner.validate(class_names=t.get_classes())